In [1667]:
import pandas as pd

In [1668]:
df_cloud_comp = pd.read_csv('cloud.csv')
df_sale = pd.read_csv('sale.csv')
df_march = pd.read_csv('march.csv')
df_may = pd.read_csv('may.csv')
df_amazon_sale = pd.read_csv('amazon.csv', low_memory = False)
df_international = pd.read_csv('international.csv')
df_expense = pd.read_csv('expense.csv')

In [1669]:
# Drop unnecessary columns
df_amazon_sale = df_amazon_sale.drop(columns=[
    'Courier Status',
    'Unnamed: 22',
    'ASIN',
    'promotion-ids',
    'ship-postal-code',
    'index'
])

# Calculate mean of Amount
mean_amount = df_amazon_sale['Amount'].mean()

# Fill missing values
df_amazon_sale = df_amazon_sale.fillna({
    'currency': 'Unknown',
    'ship-city': 'Unknown',
    'ship-state': 'Unknown',
    'ship-country': 'Unknown',
    'fulfilled-by': 'Unknown',
    'Amount': mean_amount
})

# Remove duplicates
df_amazon_sale = df_amazon_sale.drop_duplicates()

# Convert values to correct types
    # Convert 'Date' to datetime
df_amazon_sale['Date'] = pd.to_datetime(
    df_amazon_sale['Date'], 
    format='%m-%d-%y',  # Adjust if your date format differs
    errors='coerce'
)

# Check 'Qty' is numeric (integer)
df_amazon_sale['Qty'] = pd.to_numeric(df_amazon_sale['Qty'], errors='coerce') \
                              .astype('Int64') 

# Check 'Amount' is numeric (float)
df_amazon_sale['Amount'] = pd.to_numeric(df_amazon_sale['Amount'], errors='coerce')

# Check 'B2B' is boolean
df_amazon_sale['B2B'] = df_amazon_sale['B2B'].astype(bool)


In [1670]:
# Read the CSV
df_cloud_comp = pd.read_csv('cloud.csv')

# Keep only rows 1-4 where numerical cost is
df_cloud_comp = df_cloud_comp.iloc[1:5].copy()

# Rename columns:
df_cloud_comp.rename(
    columns={
        'Shiprocket': 'Description',
        'Unnamed: 1': 'Shiprocket'
    },
    inplace=True
)

# Drop columns not needed
df_cloud_comp.drop(columns=['index', 'Description'], errors='ignore', inplace=True)

# Clean and convert new 'Shiprocket' column to numeric
df_cloud_comp['Shiprocket'] = (
    df_cloud_comp['Shiprocket']
    .astype(str)
    .str.replace('₹', '', regex=False)
    .str.replace(',', '', regex=False)
    .str.extract(r'(\d+(?:\.\d+)?)')  # extract only the number pattern
    .astype(float)
)

# Clean and convert 'INCREFF' to numeric
df_cloud_comp['INCREFF'] = (
    df_cloud_comp['INCREFF']
    .astype(str)
    .str.replace('Rs', '', regex=False)
    .str.replace('/- Per Day', '', regex=False)
    .str.replace(',', '', regex=False)
    .str.extract(r'(\d+(?:\.\d+)?)')
    .astype(float)
)

# Fill NaNs with 0.0
df_cloud_comp = df_cloud_comp.fillna({'Shiprocket': 0.0, 'INCREFF': 0.0})

# Remove duplicates if any
df_cloud_comp = df_cloud_comp.drop_duplicates()

In [1671]:
# Clean international

# Read international csv
df_international = pd.read_csv('international.csv')

# Drop unnecessary columns
df_international = df_international.drop(columns=['index'], errors='ignore')

# Convert 'DATE' to datetime
df_international['DATE'] = pd.to_datetime(
    df_international['DATE'],
    errors='coerce',
    format='%m-%d-%y'  
)

# Convert numeric columns (PCS, RATE, GROSS AMT) from string/object to numeric (invalid becomes NaN for later filling)
df_international['PCS'] = pd.to_numeric(df_international['PCS'], errors='coerce')
df_international['RATE'] = pd.to_numeric(df_international['RATE'], errors='coerce')
df_international['GROSS AMT'] = pd.to_numeric(df_international['GROSS AMT'], errors='coerce')

# Fill missing values
    # For numeric columns (PCS, RATE, GROSS AMT) fill with 0
    # For text columns (Style, SKU, Size, Months, CUSTOMER) fill with 'Unknown'
df_international = df_international.fillna({
    'Style': 'Unknown',
    'SKU': 'Unknown',
    'Size': 'Unknown',
    'Months': 'Unknown',
    'CUSTOMER': 'Unknown',
    'PCS': 0,
    'RATE': 0.0,
    'GROSS AMT': 0.0
})

# Convert PCS to integer
df_international['PCS'] = df_international['PCS'].astype(int)

# Remove duplicates if any
df_international = df_international.drop_duplicates()

In [1672]:
# Clean expense

# Read csv
df_expense = pd.read_csv('expense.csv')

# Drop the first row (extra headers)
df_expense = df_expense.iloc[1:].copy()

# Drop index (pandas already has its own index)
df_expense = df_expense.drop(columns=['index'])


# Rename columns
df_expense = df_expense.rename(
    columns={
        'Recived Amount': 'Date',
        'Unnamed: 1': 'Received_Amount',
        'Expance': 'ExpenseDesc',
        'Unnamed: 3': 'ExpenseAmount'
    }
)

# Remove row that says 'Total' in the Date column (row 16)
df_expense = df_expense[df_expense['Date'] != 'Total']

# Convert Date to datetime
df_expense['Date'] = pd.to_datetime(
    df_expense['Date'],
    errors='coerce',      # invalid formats become NaT
    format='%m-%d-%y'     # update if your date format differs
)

# Convert numeric columns
df_expense['Received_Amount'] = pd.to_numeric(
    df_expense['Received_Amount'], errors='coerce'
)
df_expense['ExpenseAmount'] = pd.to_numeric(
    df_expense['ExpenseAmount'], errors='coerce'
)

# Fill missing numeric values with 0
df_expense['Received_Amount'] = df_expense['Received_Amount'].fillna(0)
df_expense['ExpenseAmount'] = df_expense['ExpenseAmount'].fillna(0)

# Remove duplicates if any
df_expense = df_expense.drop_duplicates()

In [1673]:
# Clean may

# Read csv
df_may = pd.read_csv('may.csv')

# Drop columns (index)
df_may = df_may.drop(columns=['index'], errors='ignore')

# Convert needed columns to numeric
numeric_cols = [
    'Weight',
    'MRP Old',
    'Final MRP Old',
    'Ajio MRP',
    'Amazon MRP',
    'Amazon FBA MRP',
    'Flipkart MRP',
    'Limeroad MRP',
    'Myntra MRP',
    'Paytm MRP',
    'Snapdeal MRP',
    'TP'
]

# Convert each column in numeric_cols to numeric
for col in numeric_cols:
    if col in df_may.columns:
        df_may[col] = pd.to_numeric(df_may[col], errors='coerce')

# Fill missing values:
    # Numeric columns = 0
    # String columns = 'Unknown'
string_cols = ['Sku', 'Catalog', 'Category']  

# Fill numeric columns with 0
df_may[numeric_cols] = df_may[numeric_cols].fillna(0)

# Fill string columns with 'Unknown'
for col in string_cols:
    if col in df_may.columns:
        df_may[col] = df_may[col].fillna('Unknown')

# Remove duplicates if any
df_may = df_may.drop_duplicates()

In [1674]:
# Clean march

# Read csv
df_march = pd.read_csv('march.csv')

# Drop columns (index)
df_march = df_march.drop(columns=['index'], errors='ignore')

# Convert relevant columns to numeric
numeric_cols = [
    'Weight',
    'MRP Old',
    'Final MRP Old',
    'Ajio MRP',
    'Amazon MRP',
    'Amazon FBA MRP',
    'Flipkart MRP',
    'Limeroad MRP',
    'Myntra MRP',
    'Paytm MRP',
    'Snapdeal MRP',
    'TP 1',
    'TP 2',
]

# Convert each column in numeric_cols to numeric
for col in numeric_cols:
    if col in df_march.columns:
        df_march[col] = pd.to_numeric(df_march[col], errors='coerce')

# Fill missing values:
    # Numeric columns = 0
    # String columns = 'Unknown'
string_cols = ['Sku', 'Catalog', 'Category'] 

# Fill numeric columns with 0
df_march[numeric_cols] = df_march[numeric_cols].fillna(0)

# Fill string columns with 'Unknown'
for col in string_cols:
    if col in df_march.columns:
        df_march[col] = df_march[col].fillna('Unknown')

# Remove duplicates if any
df_march = df_march.drop_duplicates()

In [1675]:
# Clean sale

# Read csv
df_sale = pd.read_csv('sale.csv')

# Drop columns (index)
df_sale = df_sale.drop(columns=['index'], errors='ignore')


# Convert 'Stock' to numeric
df_sale['Stock'] = pd.to_numeric(df_sale['Stock'], errors='coerce')

# Fill missing values
    # 'Stock' (numeric) = 0
    # String columns = 'Unknown'
string_cols = ['SKU Code', 'Design No.', 'Category', 'Size', 'Color']

df_sale['Stock'] = df_sale['Stock'].fillna(0)

for col in string_cols:
    if col in df_sale.columns:
        df_sale[col] = df_sale[col].fillna('Unknown')

# Remove duplicates
df_sale = df_sale.drop_duplicates()

In [1676]:
# Numeric Summaries
df_amazon_sale.describe()
df_cloud_comp.describe()
df_may.describe()
df_march.describe()
df_sale.describe()
df_expense.describe()
df_international.describe()

,DATE,PCS,RATE,GROSS AMT
count,12322,23887.000000,23887.000000,23887.000000
mean,2021-11-22 11:36:30.618406144,366.952736,812.732836,470.148876
min,2021-06-05 00:00:00,0.000000,0.000000,0.000000
25%,2021-09-11 00:00:00,1.000000,462.500000,10.000000
50%,2021-11-03 00:00:00,3.000000,628.000000,353.000000
75%,2022-02-24 00:00:00,562.000000,925.000000,688.000000
max,2022-05-11 00:00:00,57400.000000,57400.000000,9745.000000
std,NaN,880.992478,917.042063,638.411231


In [1677]:
# Analyze general sales trends

# Make 'Date' a datetime
df_amazon_sale['Month'] = df_amazon_sale['Date'].dt.month
df_amazon_sale['Year'] = df_amazon_sale['Date'].dt.year

# Group by Year and Month to see total Amount
monthly_sales = df_amazon_sale.groupby(['Year', 'Month'], as_index=False)['Amount'].sum()

print('Monthly Sales:')
print(monthly_sales.head())

# Sales by different category
category_sales = df_amazon_sale.groupby('Category', as_index=False)['Amount'].sum()
category_sales.sort_values('Amount', ascending=False, inplace=True)

print('Category Sales (Descending):')
print(category_sales.head())

# Currency and stock level 
print('Currency Counts:')
print(df_amazon_sale['currency'].value_counts())

# Stock level stats
print('Stock Summary:')
print(df_sale['Stock'].describe())

# Filter low-stock items
low_stock_items = df_sale[df_sale['Stock'] < 10]
print('Items with <10 in Stock:')
print(low_stock_items.head())


Monthly Sales:
   Year  Month        Amount
0  2022      3  1.075209e+05
1  2022      4  3.078245e+07
2  2022      5  2.784835e+07
3  2022      6  2.490531e+07
Category Sales (Descending):
        Category        Amount
5            Set  4.130401e+07
8          kurta  2.334717e+07
7  Western Dress  1.173233e+07
6            Top  5.644185e+06
3   Ethnic Dress  8.340227e+05
Currency Counts:
currency
INR        121177
Unknown      7792
Name: count, dtype: int64
Stock Summary:
count    9233.000000
mean       26.252139
std        58.467946
min         0.000000
25%         3.000000
50%         8.000000
75%        31.000000
max      1234.000000
Name: Stock, dtype: float64
Items with <10 in Stock:
        SKU Code Design No.  Stock       Category Size Color
0    AN201-RED-L      AN201    5.0  AN : LEGGINGS    L   Red
1    AN201-RED-M      AN201    5.0  AN : LEGGINGS    M   Red
2    AN201-RED-S      AN201    3.0  AN : LEGGINGS    S   Red
3   AN201-RED-XL      AN201    6.0  AN : LEGGINGS   XL   

In [1679]:
# Export new data to csv for Tableau

# Amazon Sale Report
df_amazon_sale.to_csv('amazon_sale_cleaned.csv', index=False)

# Cloud Warehouse Comparison Chart
df_cloud_comp.to_csv('cloud_comp_cleaned.csv', index=False)

# Expense IIGF
df_expense.to_csv('expense_cleaned.csv', index=False)

# International Sale Report
df_international.to_csv('international_cleaned.csv', index=False)

# May 2022
df_may.to_csv('may_cleaned.csv', index=False)

# March 2021
df_march.to_csv('march_cleaned.csv', index=False)

# Sale Report
df_sale.to_csv('sale_cleaned.csv', index=False)

# Additional Summary DataFrames
monthly_sales.to_csv('monthly_sales.csv', index=False)
category_sales.to_csv('category_sales.csv', index=False)



In [1680]:
# Compare Prices Across Channels

# Descriptive analysis
channel_cols = ['Amazon FBA MRP', 'Myntra MRP', 'Ajio MRP', 
                'Flipkart MRP', 'Paytm MRP', 'Snapdeal MRP']
channel_stats = df_may[channel_cols].describe()
print('Channel MRP Statistics:')
print(channel_stats)

# Identify Which Channel Has Highest/Lowest Prices
avg_mrp = df_may[channel_cols].mean().sort_values(ascending=False)
print('Average MRP by Channel (Descending):')
print(avg_mrp)

Channel MRP Statistics:
       Amazon FBA MRP   Myntra MRP     Ajio MRP  Flipkart MRP    Paytm MRP  \
count     1330.000000  1330.000000  1330.000000   1330.000000  1330.000000   
mean      2185.256729  2180.483045  2178.913120   2180.717632  2176.215376   
std        786.556700   763.422217   789.392891    787.270554   784.746164   
min          0.000000     0.000000     0.000000      0.000000     0.000000   
25%       1795.000000  1795.000000  1695.000000   1695.000000  1795.000000   
50%       2095.000000  2095.000000  2050.000000   2095.000000  2050.000000   
75%       2495.000000  2494.000000  2494.000000   2494.000000  2494.000000   
max       5997.000000  5997.000000  5997.000000   5997.000000  5997.000000   

       Snapdeal MRP  
count   1330.000000  
mean    2177.418383  
std      785.013724  
min        0.000000  
25%     1695.000000  
50%     2095.000000  
75%     2494.000000  
max     5997.000000  
Average MRP by Channel (Descending):
Amazon FBA MRP    2185.256729
Flipkart

In [1681]:
# Customer-Specific Data 

# Group by SKU & CUSTOMER to see total GROSS AMT
sku_customer_sales = df_international.groupby(['SKU','CUSTOMER'], as_index=False)['GROSS AMT'].sum()
sku_customer_sales.sort_values('GROSS AMT', ascending=False, inplace=True)
print('Gross Amount by SKU & Customer:')
print(sku_customer_sales.head(15))

# Analyzing over time
df_international['Year'] = df_international['DATE'].dt.year
df_international['Month'] = df_international['DATE'].dt.month

# sum GROSS AMT monthly per SKU
sku_monthly = df_international.groupby(['SKU','Year','Month'], as_index=False)['GROSS AMT'].sum()
print('SKU Performance Over Time:')
print(sku_monthly.head(10))


Gross Amount by SKU & Customer:
           SKU                         CUSTOMER  GROSS AMT
19742  Unknown              MULBERRIES BOUTIQUE   380453.5
19721  Unknown          BHANU SALEINE NAUNITHAM    71675.0
19736  Unknown                Jeannie Thein Win    71400.0
19730  Unknown                     Diya Designs    69270.0
19732  Unknown  Fusion Fashions Corp. (Gopikas)    67840.0
19734  Unknown                          Gajanan    59887.5
19725  Unknown                   Chaaya Sookdeb    54725.0
19747  Unknown                   Priyanka Gupta    53260.0
19761  Unknown                  The Pink Market    37890.0
19745  Unknown                        Natheliya    35675.0
19746  Unknown        Nitha Pushpangadan Sarala    32300.0
19753  Unknown                    Rino Sandaran    30347.5
19752  Unknown                      Ria Fashion    29735.1
19763  Unknown            Valli Arangan Fashion    29162.5
19755  Unknown                             Siva    27199.0
SKU Performance Over Tim

In [1682]:
# Overall Stock + Expenses for Cost-Cutting Trends 

# Summarize expenses
total_received = df_expense['Received_Amount'].sum()
total_expense = df_expense['ExpenseAmount'].sum()
net_balance = total_received - total_expense

print('Total Received:', total_received)
print('Total Expenses:', total_expense)
print('Net Balance:', net_balance)

# Merge Stock with Sales or P&L
df_stock_pl = pd.merge(
    df_sale[['SKU Code','Stock']],
    df_march,         
    left_on='SKU Code',
    right_on='Sku',          
    how='left'
)

print('Merged Stock & P&L Data:')
print(df_stock_pl.head())

# Identify High Stock, Low Turnover
df_merged_stock_sales = pd.merge(
    df_sale[['SKU Code','Stock']],
    df_amazon_sale.groupby('SKU', as_index=False)['Qty'].sum(), 
    left_on='SKU Code',
    right_on='SKU',
    how='left'
)

df_merged_stock_sales['Qty'] = df_merged_stock_sales['Qty'].fillna(0)

# Stock turnover ratio or leftover stock
df_merged_stock_sales['Leftover'] = df_merged_stock_sales['Stock'] - df_merged_stock_sales['Qty']

high_leftover = df_merged_stock_sales[df_merged_stock_sales['Leftover'] > 50]
print('SKUs with leftover > 50:')
print(high_leftover.head())


Total Received: 5000.0
Total Expenses: 13095
Net Balance: -8095.0
Merged Stock & P&L Data:
        SKU Code  Stock  Sku Style Id Catalog Category  Weight  TP 1  TP 2  \
0    AN201-RED-L    5.0  NaN      NaN     NaN      NaN     NaN   NaN   NaN   
1    AN201-RED-M    5.0  NaN      NaN     NaN      NaN     NaN   NaN   NaN   
2    AN201-RED-S    3.0  NaN      NaN     NaN      NaN     NaN   NaN   NaN   
3   AN201-RED-XL    6.0  NaN      NaN     NaN      NaN     NaN   NaN   NaN   
4  AN201-RED-XXL    3.0  NaN      NaN     NaN      NaN     NaN   NaN   NaN   

   MRP Old  Final MRP Old  Ajio MRP  Amazon MRP  Amazon FBA MRP  Flipkart MRP  \
0      NaN            NaN       NaN         NaN             NaN           NaN   
1      NaN            NaN       NaN         NaN             NaN           NaN   
2      NaN            NaN       NaN         NaN             NaN           NaN   
3      NaN            NaN       NaN         NaN             NaN           NaN   
4      NaN            NaN       NaN

In [1683]:
# Group by SKU, Customer, and Date
sku_customer_summary = df_amazon_sale.groupby(['SKU', 'Order ID', 'Date']).agg({
    'Amount': 'sum',  
    'Qty': 'sum'  
}).reset_index()

# Save to CSV
sku_customer_summary.to_csv('sku_customer_summary.csv', index=False)


In [1684]:
# Merge stock and sales data
category_stock_sales = df_sale.merge(
    df_amazon_sale, on='Category', how='inner'
)

# Group by category
category_analysis = category_stock_sales.groupby('Category').agg({
    'Stock': 'sum',      
    'Amount': 'sum'     
}).reset_index()

# Calculate Stock-to-Sales Ratio
category_analysis['Stock-to-Sales Ratio'] = category_analysis['Stock'] / category_analysis['Amount']

# Save to CSV for Tableau
category_analysis.to_csv('category_stock_sales.csv', index=False)


In [1685]:
# Display unique categories in both DataFrames
print('Unique categories in df_salet:')
print(df_sale['Category'].unique())

print('\nUnique categories in df_amazon_sale:')
print(df_amazon_sale['Category'].unique())


Unique categories in df_salet:
['AN : LEGGINGS' 'BLOUSE' 'PANT' 'BOTTOM' 'PALAZZO' 'SHARARA' 'SKIRT'
 'DRESS' 'KURTA SET' 'LEHENGA CHOLI' 'SET' 'TOP' 'KURTA' 'Unknown'
 'CROP TOP' 'TUNIC' 'CARDIGAN' 'JUMPSUIT' 'CROP TOP WITH PLAZZO' 'SAREE'
 'KURTI' 'NIGHT WEAR']

Unique categories in df_amazon_sale:
['Set' 'kurta' 'Western Dress' 'Top' 'Ethnic Dress' 'Bottom' 'Saree'
 'Blouse' 'Dupatta']


In [1686]:

# Ensure Date column is in datetime format
df_amazon_sale['Date'] = pd.to_datetime(df_amazon_sale['Date'], errors='coerce')

# Group by Date and calculate total sales
sales_over_time = df_amazon_sale.groupby('Date')['Amount'].sum().reset_index()

# Save aggregated data to a CSV for Tableau
sales_over_time.to_csv('sales_over_time.csv', index=False)

# Print first few rows to verify
print(sales_over_time.head())


        Date        Amount
0 2022-03-31  1.075209e+05
1 2022-04-01  9.355232e+05
2 2022-04-02  9.773091e+05
3 2022-04-03  1.075971e+06
4 2022-04-04  9.410783e+05


In [1687]:
# Load cleaned May CSV
df_may = pd.read_csv('may_cleaned.csv')

# Pivot the MRP columns into 'Channel' and 'MRP' for analysis
channel_data = df_may.melt(
    id_vars=['Sku', 'Style Id', 'Catalog', 'Category', 'Weight', 'TP', 'MRP Old', 'Final MRP Old'],  # Columns to retain
    value_vars=[
        'Ajio MRP', 'Amazon MRP', 'Amazon FBA MRP', 'Flipkart MRP', 
        'Limeroad MRP', 'Myntra MRP', 'Paytm MRP', 'Snapdeal MRP'
    ],  # Columns to pivot
    var_name='Channel',   # New column for channel names
    value_name='MRP'      # New column for the MRP values
)

# Save pivoted data to a CSV for Tableau
channel_data.to_csv('channel_comparison.csv', index=False)

# Preview first few rows to confirm
print(channel_data.head())


              Sku    Style Id  Catalog Category  Weight     TP  MRP Old  \
0    Os206_3141_S  Os206_3141  Moments    Kurta     0.3  538.0   2178.0   
1    Os206_3141_M  Os206_3141  Moments    Kurta     0.3  538.0   2178.0   
2    Os206_3141_L  Os206_3141  Moments    Kurta     0.3  538.0   2178.0   
3   Os206_3141_XL  Os206_3141  Moments    Kurta     0.3  538.0   2178.0   
4  Os206_3141_2XL  Os206_3141  Moments    Kurta     0.3  538.0   2178.0   

   Final MRP Old   Channel     MRP  
0         2295.0  Ajio MRP  2295.0  
1         2295.0  Ajio MRP  2295.0  
2         2295.0  Ajio MRP  2295.0  
3         2295.0  Ajio MRP  2295.0  
4         2295.0  Ajio MRP  2295.0  
